In [24]:
import torch
import gradientUtils as gu
# Define a threshold for early stopping
threshold = 1e-6
circle_center = torch.tensor([5.0, 5.0])  # Example circle center
radius = 3.0  # Example radius
alpha = 0.1  # Learning rate
delta = 1e-5

In [25]:
circle_center = torch.tensor([0.0, 0.0])  # Example circle center
radius = 1.0  # Example radius


# Initialize the source points (p1, p2, p3) as PyTorch tensors
p1 = torch.tensor([2.0, 2.0], requires_grad=True)
p2 = torch.tensor([-2.01, 2.0], requires_grad=True)
p3 = torch.tensor([-20.0, -2.0], requires_grad=True)
destination = "images/autograd/cccsdf/"
prev_loss = float("inf")


# Define the optimizer
optimizer = torch.optim.Adam([p1, p2, p3], lr=0.1)

# Optimization loop
for step in range(100000):
    optimizer.zero_grad()
    
    # Compute the point (x, y) from the source points and weights
    x,y = gu.compute_vertex(p1, p2, p3)
    
    # Calculate the SDF value at the current point
    vertex_loss = gu.circle_sdf(x,y, circle_center, radius)**2
    # Calculate the SDF values of the midpoints
    midpoint_loss = gu.midpoint_loss_sdf(p1, p2, p3, circle_center, radius)
    # Define the loss as the squared distance
    loss = vertex_loss + 0.005 * midpoint_loss
    
    # Early stopping condition
    if abs(prev_loss - loss.item()) < threshold:
        print(f"Stopping early at epoch {step} due to minimal loss change")
        gu.plot_and_save(
            step, p1, p2, p3, circle_center, radius, destination=destination
        )
        break

    prev_loss = loss.item()
    # Backpropagation
    loss.backward()
    
    # Optimization step
    optimizer.step()
    
    # Print progress every 100 steps
    if step % 100 == 0:
        print(f'Step {step}, Point: {x.item(),y.item()}, Loss: {loss.item()}, distance: {vertex_loss.item()}')
        gu.plot_and_save(step, p1, p2, p3,circle_center, radius, destination=destination)


# Final optimized point
x,y = gu.compute_vertex(p1, p2, p3)
print(f'Optimized Point: {x.item(),y.item()}')
print(f'Optimized p1: {p1.detach().numpy()}')
print(f'Optimized p2: {p2.detach().numpy()}')
print(f'Optimized p3: {p3.detach().numpy()}')
gu.plot_and_save(step, p1, p2, p3, circle_center, radius, destination=destination)




Step 0, Point: (-0.005000011995434761, -49.47249984741211), Loss: 2350.0888671875, distance: 2349.583251953125
Step 100, Point: (-8.370285034179688, -2.441863536834717), Loss: 59.934871673583984, distance: 59.585975646972656
Step 200, Point: (-7.206372261047363, -2.0910027027130127), Loss: 42.551536560058594, distance: 42.29688262939453
Step 300, Point: (-5.929025650024414, -1.76790189743042), Loss: 27.081539154052734, distance: 26.904848098754883
Step 400, Point: (-4.730807781219482, -1.439761757850647), Loss: 15.691061973571777, distance: 15.563368797302246
Step 500, Point: (-3.698775053024292, -1.140509009361267), Loss: 8.346173286437988, distance: 8.240457534790039
Step 600, Point: (-2.8663158416748047, -0.8902053833007812), Loss: 4.107691764831543, distance: 4.005488395690918
Step 700, Point: (-2.232128381729126, -0.6957085728645325), Loss: 1.8985053300857544, distance: 1.790337324142456
Step 800, Point: (-1.7741292715072632, -0.5540570616722107), Loss: 0.8542945384979248, distanc